https://qiita.com/opeco17/items/707a5c57bca41a145122

In [1]:
import numpy as np
import math
import random
import os
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F


In [2]:
class NonLinear(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(NonLinear, self).__init__()
        self.input_channels = input_channels
        self.output_channels = output_channels

        self.main = nn.Sequential(
            nn.Linear(self.input_channels, self.output_channels),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(self.output_channels))

    def forward(self, input):
        return self.main(input)



In [3]:
class MaxPool(nn.Module):
    def __init__(self, num_channels, num_points):
        super(MaxPool, self).__init__()
        self.num_channels = num_channels
        self.num_points = num_points
        self.main = nn.MaxPool1d(self.num_points)
    
    def forward(self, input):
        out = input.view(-1, self.num_channels, self.num_points)
        out = self.main(out)
        out = out.view(-1, self.num_channels)

        return out

In [4]:
class InputTNet(nn.Module):
    def __init__(self, num_points):
        super().__init__()
        self.num_points = num_points

        self.main = nn.Sequential(
            NonLinear(3, 64),
            NonLinear(64, 128),
            NonLinear(128, 1024),
            MaxPool(1024, self.num_points),
            NonLinear(1024, 512),
            NonLinear(512, 256),
            nn.Linear(256, 9)
        )

    def forward(self, input):
        matrix = self.main(input).view(-1, 3, 3)
        out = torch.matmul(input.view(-1, self.num_points, 3), matrix)
        out = out.view(-1, 3)

        return out

In [5]:
class FeatureTNet(nn.Module):
    def __init__(self, num_points):
        super(FeatureTNet, self).__init__()
        self.num_points = num_points

        self.main = nn.Sequential(
            NonLinear(64, 64),
            NonLinear(64, 128),
            NonLinear(128, 1024),
            MaxPool(1024, self.num_points),
            NonLinear(1024, 512),
            NonLinear(512, 256),
            nn.Linear(256, 4096)
        )
    
    def forward(self, input):
        matrix = self.main(input).view(-1, 64, 64)
        out = torch.matmul(input.view(-1, self.num_points, 64), matrix)
        out = out.view(-1, 64)

        return out

In [6]:
class PointNet(nn.Module):
    def __init__(self, num_points, num_labels):
        super(PointNet, self).__init__()
        self.num_points = num_points
        self.num_labels = num_labels

        self.main = nn.Sequential(
            InputTNet(self.num_points),
            NonLinear(3, 64),
            NonLinear(64, 64),
            FeatureTNet(self.num_points),
            NonLinear(64, 64),
            NonLinear(64, 128),
            NonLinear(128, 1024),
            MaxPool(1024, self.num_points),
            NonLinear(1024, 512),
            nn.Dropout(p=0.3),
            NonLinear(512, 256),
            nn.Dropout(p=0.3),
            NonLinear(256, self.num_labels)
        )

    def forward(self, input):
        return self.main(input)

In [7]:
def data_sampler(batch_size, num_points):
    half_batch_size = int(batch_size/2)
    normal_sampled = torch.randn(half_batch_size, num_points, 3)
    uniform_sampled = torch.rand(half_batch_size, num_points, 3)
    normal_labels = torch.ones(half_batch_size)
    uniform_labels = torch.zeros(half_batch_size)

    input_data = torch.cat((normal_sampled, uniform_sampled), dim=0)
    labels = torch.cat((normal_labels, uniform_labels), dim=0)

    data_shuffle = torch.randperm(batch_size)

    return input_data[data_shuffle].view(-1, 3), labels[data_shuffle].view(-1, 1)

In [8]:
# Main function

batch_size = 64
num_points = 64
num_labels = 1

pointnet = PointNet(num_points, num_labels)

new_param = pointnet.state_dict()
new_param['main.0.main.6.bias'] = torch.eye(3, 3).view(-1)
new_param['main.3.main.6.bias'] = torch.eye(64, 64).view(-1)
pointnet.load_state_dict(new_param)

criterion = nn.BCELoss()
optimizer = optim.Adam(pointnet.parameters(), lr=0.001)

loss_list = []
accuracy_list = []

for iteration in range(100+1):

    pointnet.zero_grad()
    input, labels = data_sampler(batch_size, num_points)

    output = pointnet(input)
    output = nn.Sigmoid()(output)

    error = criterion(output, labels)
    error.backward()

    optimizer.step()

    with torch.no_grad():
        output[output > 0.5] = 1
        output[output < 0.5] = 0
        accuracy = (output==labels).sum().item()/batch_size

    loss_list.append(error.item())
    accuracy_list.append(accuracy)

    if iteration % 10 == 0:
        print(f'Iteration: {iteration}    Loss: {error.item()}')
        print(f'Iteration: {iteration}    Accuracy: {accuracy}')

Iteration: 0    Loss: 0.6403641700744629
Iteration: 0    Accuracy: 0.578125
Iteration: 10    Loss: 0.34747737646102905
Iteration: 10    Accuracy: 1.0
Iteration: 20    Loss: 0.3266719877719879
Iteration: 20    Accuracy: 1.0
Iteration: 30    Loss: 0.3157070279121399
Iteration: 30    Accuracy: 1.0
Iteration: 40    Loss: 0.31111931800842285
Iteration: 40    Accuracy: 1.0
Iteration: 50    Loss: 0.3056040406227112
Iteration: 50    Accuracy: 1.0
Iteration: 60    Loss: 0.3026292026042938
Iteration: 60    Accuracy: 1.0
Iteration: 70    Loss: 0.3026331663131714
Iteration: 70    Accuracy: 1.0
Iteration: 80    Loss: 0.299703449010849
Iteration: 80    Accuracy: 1.0
Iteration: 90    Loss: 0.29401636123657227
Iteration: 90    Accuracy: 1.0
Iteration: 100    Loss: 0.28994905948638916
Iteration: 100    Accuracy: 1.0


In [9]:
input, labels = data_sampler(batch_size, num_points)
print(f'input.size(): {input.size()}')
print(f'labels.size(): {labels.size()}')

input.size(): torch.Size([4096, 3])
labels.size(): torch.Size([64, 1])
